# McStasScript
McStasScript allows writing and running McStas simulations directly from python.

Written by Mads Bertelsen while working at ESS DMSC.

Funded by PaNOSC and a part of PaNOSC work package 5.

Documentation online: https://mads-bertelsen.github.io

### Installation
McStasScript is open source and available here: https://github.com/PaNOSC-ViNYL/McStasScript

Easy to install, install McStas as normal and then McStasScript can be installed with:
```
pip install McStasScript --upgrade
```

![normal](figures/Slide1.png)

![McStasScript](figures/Slide2.png)

## Why move to Python?
- McStas can still be used as usual, McStasScript is just an alternative
- New generation of scientists are more likely to know Python
- One language for writing the simulation and analysis of data

## Why have a software layer that manages McStas things?
- Provides powerful levels of abstraction (functions, for loops, ...)
- Can in many cases find problems earlier than when running the instrument
- Can host relevant McStas tools

# Live demo of McStasScript
Here we will set up an instrument simulation together and execute it from the slides!
First we import the package.

In [1]:
import mcstasscript as ms

## Configuration
Before using McStasScript for the first time, it needs to be configured to find your McStas and/or McXtrace installation.

In [2]:
configurator = ms.Configurator()
configurator.set_mcrun_path("/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/")
configurator.set_mcstas_path("/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1")
print(configurator)

Configurator:
 paths:
  mcrun_path: /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/
  mcstas_path: /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1
  mcxtrace_path: /Applications/McXtrace-3.0.app/Contents/Resources/mcxtrace/3.0/
  mxrun_path: /Applications/McXtrace-3.0.app/Contents/Resources/mcxtrace/3.0/bin/
 other:
  characters_per_line: 85



## Create instrument objects
The main class is the McStas_instr, used for creating instrument objects.

In [3]:
instrument = ms.McStas_instr("demo", author="Mads Bertelsen")

## Help from the instrument object
The instrument object knows about your available McStas components, and has help methods available.

In [4]:
instrument.available_components()

Here are the available component categories:
 contrib
 misc
 monitors
 obsolete
 optics
 samples
 sources
 union
Call available_components(category_name) to display


In [5]:
instrument.available_components("sources")

Here are all components in the sources category.
 Adapt_check     Moderator           Source_Optimizer   Source_gen
 ESS_butterfly   Monitor_Optimizer   Source_adapt       Source_simple
 ESS_moderator   Source_Maxwell_3    Source_div         


In [6]:
instrument.component_help("Moderator")

 ___ Help Moderator _________________________________________________________________
|optional parameter|required parameter|default value|user specified value|
radius = 0.07 [m] // Radius of source
Emin [meV] // Lower edge of energy distribution
Emax [meV] // Upper edge of energy distribution
dist = 0.0 [m] // Distance from source to the focusing rectangle
focus_xw = 0.02 [m] // Width of focusing rectangle
focus_yh = 0.02 [m] // Height of focusing rectangle
t0 = 37.15 [mus] // decay constant for low-energy neutrons
Ec = 9.0 [meV] // Critical energy, below which the flux decay is constant
gamma = 39.1 [meV] // energy dependence of decay time
target_index = 1 [1] // relative index of component to focus at, e.g. next is 
                        +1 this is used to compute 'dist' automatically. 
flux = 1.0 [1/(s cm 2 st meV)] // flux
-------------------------------------------------------------------------------------


## Adding components
McStas simulations rely on a sequence of components, these can be added to the instrument object with the *add_component* method. These return an instrument object that can be used to modify the component further.

In [7]:
src = instrument.add_component("source", "Source_div")
instrument.show_components()

source Source_div AT (0, 0, 0) ABSOLUTE


In [8]:
print(src)

COMPONENT source = Source_div(
  xwidth : Required parameter not yet specified
  yheight : Required parameter not yet specified
  focus_aw : Required parameter not yet specified
  focus_ah : Required parameter not yet specified
)
AT (0, 0, 0) ABSOLUTE


The component parameters become python attributes on the component object.

In [9]:
src.xwidth = 0.1
src.yheight = 0.05
src.focus_aw = 1.2
src.focus_ah = 2.3
src.set_parameters(xwidth=0.09, yheight=0.04)
print(src)

COMPONENT source = Source_div(
  xwidth = 0.09, // [m]
  yheight = 0.04, // [m]
  focus_aw = 1.2, // [deg]
  focus_ah = 2.3 // [deg]
)
AT (0, 0, 0) ABSOLUTE


Notice the attributes on component objects auto complete and wrong attributes are not allowed.

In [10]:
src.focus_ah = 2.2

### All information about a component
The *show_parameters* method is good for getting an overview of the current state of the component parameters, as it will also show the default values.

In [11]:
src.show_parameters()

 ___ Help Source_div ________________________________________________________________
|optional parameter|required parameter|default value|user specified value|
xwidth = 0.09 [m] // Width of source
yheight = 0.04 [m] // Height of source
focus_aw = 1.2 [deg] // FWHM (Gaussian) or maximal (uniform) horz. width 
                        divergence 
focus_ah = 2.2 [deg] // FWHM (Gaussian) or maximal (uniform) vert. height 
                        divergence 
E0 = 0.0 [meV] // Mean energy of neutrons.
dE = 0.0 [meV] // Energy half spread of neutrons.
lambda0 = 0.0 [Ang] // Mean wavelength of neutrons (only relevant for E0=0)
dlambda = 0.0 [Ang] // Wavelength half spread of neutrons.
gauss = 0.0 [0|1] // Criterion
flux = 1.0 [1/(s cm 2 st energy_unit)] // flux per energy unit, Angs or meV
-------------------------------------------------------------------------------------


## Add instrument parameters
Its also possible to add instrument parameters, and these can be used directly when setting component parameters.

In [12]:
instrument.add_parameter("int", "order", value=1, comment="Monochromator order, integer")
instrument.show_parameters()

int order  = 1  // Monochromator order, integer


In [13]:
src.lambda0 = instrument.add_parameter("wavelength", value=2.0, comment="Wavelength in [Ang]")
src.dlambda = "0.01*wavelength"
print(src)

COMPONENT source = Source_div(
  xwidth = 0.09, // [m]
  yheight = 0.04, // [m]
  focus_aw = 1.2, // [deg]
  focus_ah = 2.2, // [deg]
  lambda0 = wavelength, // [Ang]
  dlambda = 0.01*wavelength // [Ang]
)
AT (0, 0, 0) ABSOLUTE


### Setting component positions
When creating a new component, one can specify its position and rotation. 

In [14]:
guide = instrument.add_component("guide", "Guide_gravity", AT=[0,0,1], RELATIVE=src)

In [15]:
guide.set_parameters(w1=0.05, h1=0.05, l=8.0, m=3.5, G=-9.82)
print(guide)

COMPONENT guide = Guide_gravity(
  w1 = 0.05, // [m]
  h1 = 0.05, // [m]
  l = 8.0, // [m]
  m = 3.5, // [1]
  G = -9.82 // [m/s2]
)
AT (0, 0, 1) RELATIVE source


One can always set the positon and rotation of component objects with the appropriate methods at a later point.

In [16]:
guide.set_AT([0, 0, 2], RELATIVE="source")

### Adding initialize code
Its possible to add lines of C code to the initialize section of the written McStas file, but its equally possible to just perform these calculations directly in python. Here I show how calculate rotation of a monochromator before we add such a component.

In [17]:
instrument.add_declare_var("double", "mono_Q", value=2*3.14159/1.714) # Q for Ge 311
instrument.add_declare_var("double", "wavevector")
instrument.append_initialize("wavevector = 2.0*PI/wavelength;")

instrument.add_declare_var("double", "mono_rotation")
instrument.append_initialize("mono_rotation = asin(mono_Q/(2.0*wavevector))*RAD2DEG;")

In [64]:
instrument.show_variables()

DECLARE VARIABLES 
type    variable name  array length  value              
------------------------------------------------------
double  mono_Q                       3.665799299883314  
double  wavevector                                      
double  mono_rotation                                   



### Adding a monochromator
We can now add a monochromator that use the calculated rotation.

In [18]:
mono = instrument.add_component("mono", "Monochromator_flat")

In [19]:
mono.set_parameters(zwidth=0.05, yheight=0.08, Q="mono_Q")
mono.set_AT([0, 0, guide.l + 0.5], RELATIVE=guide)
mono.set_ROTATED([0, "mono_rotation", 0], RELATIVE="guide")

print(mono)

COMPONENT mono = Monochromator_flat(
  zwidth = 0.05, // [m]
  yheight = 0.08, // [m]
  Q = mono_Q // [1/angstrom]
)
AT (0, 0, 8.5) RELATIVE guide
ROTATED (0, 'mono_rotation', 0) RELATIVE guide


Need to set the beam direction to the direction of the scattered beam

In [20]:
beam_direction = instrument.add_component("beam_dir", "Arm", AT_RELATIVE=mono)
beam_direction.set_ROTATED([0, "mono_rotation", 0], RELATIVE=mono)

### Adding a sample
Now we can set up a simple powder sample to get some mildly interesting data.

In [21]:
sample = instrument.add_component("sample", "PowderN", AT=[0,0,1.1], RELATIVE="beam_dir")

In [22]:
sample.radius = 0.015
sample.yheight = 0.05
sample.reflections = '"Na2Ca3Al2F14.laz"'
print(sample)

COMPONENT sample = PowderN(
  reflections = "Na2Ca3Al2F14.laz", // []
  radius = 0.015, // [m]
  yheight = 0.05 // [m]
)
AT (0, 0, 1.1) RELATIVE beam_dir


### Adding a few monitors
At the end we add a few monitors, a two theta banana detector and a transmission PSD.

In [23]:
banana = instrument.add_component("banana", "Monitor_nD", RELATIVE=sample)
banana.xwidth = 2.0
banana.yheight = 0.3
banana.restore_neutron = 1
banana.filename = '"banana.dat"'
banana.options = '"theta limits=[5 175] bins=150, banana"'

mon = instrument.add_component("monitor", "PSD_monitor")
mon.nx = 100
mon.ny = 100
mon.filename = '"psd.dat"'
mon.xwidth = 0.05
mon.yheight = 0.08
mon.restore_neutron = 1
mon.set_AT([0,0,0.1], RELATIVE=sample)

### Checking our instrument object
Before running our simulation we can check the contents of the instrument object and ensure its reasonable.

In [24]:
instrument.show_components()

source   Source_div         AT      (0, 0, 0)             ABSOLUTE         
guide    Guide_gravity      AT      (0, 0, 2)             RELATIVE source  
mono     Monochromator_flat AT      (0, 0, 8.5)           RELATIVE guide    
                            ROTATED (0, mono_rotation, 0) RELATIVE guide
beam_dir Arm                AT      (0, 0, 0)             RELATIVE mono     
                            ROTATED (0, mono_rotation, 0) RELATIVE mono
sample   PowderN            AT      (0, 0, 1.1)           RELATIVE beam_dir
banana   Monitor_nD         AT      (0, 0, 0)             RELATIVE sample  
monitor  PSD_monitor        AT      (0, 0, 0.1)           RELATIVE sample  


In [25]:
instrument.show_parameters()

int order       = 1    // Monochromator order, integer
    wavelength  = 2.0  // Wavelength in [Ang]


### Instrument diagram
Instrument diagram can be generated to provide an overview of the instrument.

In [26]:
%matplotlib widget
instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Visualizing the instrument
Can visualize the instrument directly from the notebook

In [27]:
instrument.show_instrument(width=1000,height=800)

### Runnig our simulation
Now we can run the simulation directly from python.

In [28]:
instrument.settings(ncount=5E6, output_path="data_folder/mcstas_basics")
instrument.set_parameters(wavelength=2.8)
data = instrument.backengine()

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).
Opening input file '/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1//data/Na2Ca3Al2F14.laz' (Table_Read_Offset)
Table from file 'Na2Ca3Al2F14.laz' (block 1) is 841 x 18 (x=1:20), constant step. interpolation: linear
  '# TITLE *-Na2Ca3Al2F14-[I213] Courbion, G.;Ferey, G.[1988] Standard NAC cal ...'
PowderN: sample: Reading 841 rows from Na2Ca3Al2F14.laz
PowderN: sample: Read 841 reflections from file 'Na2Ca3Al2F14.laz'
PowderN: sample: Vc=1079.1 [Angs] sigma_abs=11.7856 [barn] sigma_inc=13.6704 [barn] reflections=Na2Ca3Al2F14.laz
Detector: banana_I=1.04189e-06 banana_ERR=1.5785e-08 banana_N=12935 "banana.dat"
Detector: monitor_I=5.33123e-05 monitor_ERR=2.26343e-07 monitor_N=125575 "psd.dat"
PowderN: sample: Info: you may highly improve the computation efficiency by using
-------------------------------------------------------------------

### Plotting the data
The *run_full_instrument* method returns a data object that we can plot.

In [29]:
print(data)

[
McStasData: banana type: 1D  I:1.04189e-06 E:1.5785e-08 N:12935.0, 
McStasData: monitor type: 2D  I:5.33123e-05 E:2.26343e-07 N:125575.0]


In [30]:
ms.make_sub_plot(data, fontsize=16, figsize=(15, 6))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting interface
A widget interface is also available for plotting.

In [31]:
%matplotlib widget
import mcstasscript.jb_interface as ms_widget
ms_widget.show(data)

### Simulation interface

In [32]:
ms_widget.show(instrument)

### Setting plotting options
Its also possible to adjust the way data is plotted, and these preferences are stored in the data object.

In [33]:
ms.name_plot_options("banana", data, left_lim=90, right_lim=150)
ms.name_plot_options("monitor", data, log=True, orders_of_mag=3)
ms.make_sub_plot(data, fontsize=12, figsize=(9, 4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Access to the underlying data
The underlying data is read from the McStas data files and imported as numpy arrays.

In [34]:
import numpy as np
banana = ms.name_search("banana", data)
print(banana)

McStasData: banana type: 1D  I:1.04189e-06 E:1.5785e-08 N:12935.0


In [35]:
print(banana.Intensity[1:10])

[1.03438897e-09 7.65047616e-10 1.23044819e-09 8.39612902e-10
 1.29966006e-09 5.20001621e-10 7.84019821e-10 7.12611654e-10
 8.84999009e-10]


### Plot with matplotlib

In [38]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7,4))
plt.plot(banana.xaxis, banana.Intensity)
plt.xlabel("Two theta [deg]")
plt.ylabel("Intensity [n/s/bin]")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Changing the instrument
The most common way to create the instrument is to add components in a sequence, but it is possible to insert a component in another position in the instrument. Lets add a slit and monitor between the monochromator and sample.

In [39]:
instrument.show_diagram()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Inserting a component before or after another
When adding a component one can specify the before or after keyword to select the location.

In [40]:
instrument.get_component("sample")

COMPONENT sample = PowderN(
  reflections = "Na2Ca3Al2F14.laz", // []
  radius = 0.015, // [m]
  yheight = 0.05 // [m]
)
AT (0, 0, 1.1) RELATIVE beam_dir

In [47]:
slit = instrument.add_component("sample_slit", "Slit", before="sample")
slit.radius = 0.01
slit.set_AT(1.0, RELATIVE="beam_dir")

mon = instrument.add_component("sample_monitor", "PSD_monitor", after=slit)
mon.set_parameters(xwidth=0.021, yheight=0.021,
                   restore_neutron=1, filename='"sample_psd"')
mon.set_AT([0,0,0.01], RELATIVE=slit)

In [45]:
instrument.show_components()

source         Source_div         AT      (0, 0, 0)             ABSOLUTE            
guide          Guide_gravity      AT      (0, 0, 2)             RELATIVE source     
mono           Monochromator_flat AT      (0, 0, 8.5)           RELATIVE guide       
                                  ROTATED (0, mono_rotation, 0) RELATIVE guide
beam_dir       Arm                AT      (0, 0, 0)             RELATIVE mono        
                                  ROTATED (0, mono_rotation, 0) RELATIVE mono
sample_slit    Slit               AT      (0, 0, 1.0)           RELATIVE beam_dir   
sample_monitor PSD_monitor        AT      (0, 0, 0.01)          RELATIVE sample_slit
sample         PowderN            AT      (0, 0, 1.1)           RELATIVE beam_dir   
banana         Monitor_nD         AT      (0, 0, 0)             RELATIVE sample     
monitor        PSD_monitor        AT      (0, 0, 0.1)           RELATIVE sample     


## Run with the slit

In [48]:
ms_widget.show(instrument)

## Moving a component
Components can be moved within the component sequence, for example we could switch the position of the monitor and slit.

In [49]:
instrument.show_components()

source         Source_div         AT      (0, 0, 0)             ABSOLUTE            
guide          Guide_gravity      AT      (0, 0, 2)             RELATIVE source     
mono           Monochromator_flat AT      (0, 0, 8.5)           RELATIVE guide       
                                  ROTATED (0, mono_rotation, 0) RELATIVE guide
beam_dir       Arm                AT      (0, 0, 0)             RELATIVE mono        
                                  ROTATED (0, mono_rotation, 0) RELATIVE mono
sample_slit    Slit               AT      (0, 0, 1.0)           RELATIVE beam_dir   
sample_monitor PSD_monitor        AT      (0, 0, 0.01)          RELATIVE sample_slit
sample         PowderN            AT      (0, 0, 1.1)           RELATIVE beam_dir   
banana         Monitor_nD         AT      (0, 0, 0)             RELATIVE sample     
monitor        PSD_monitor        AT      (0, 0, 0.1)           RELATIVE sample     


In [50]:
instrument.move_component("sample_slit", after="sample_monitor")

Moving the component 'sample_slit' introduced errors in the instrument, run check_for_errors() for more information.


### Checking the instrument for errors
Some operations checks for errors, for example moving a component, one can also do it manually and get the error message.

In [54]:
try:
    instrument.check_for_errors()
except Exception as e:
    print(e)

Component 'sample_monitor' referenced unknown component named 'sample_slit'.
This check can be skipped with settings(checks=False)


### Fixing the instrument

In [53]:
instrument.show_components()

source         Source_div         AT      (0, 0, 0)             ABSOLUTE            
guide          Guide_gravity      AT      (0, 0, 2)             RELATIVE source     
mono           Monochromator_flat AT      (0, 0, 8.5)           RELATIVE guide       
                                  ROTATED (0, mono_rotation, 0) RELATIVE guide
beam_dir       Arm                AT      (0, 0, 0)             RELATIVE mono        
                                  ROTATED (0, mono_rotation, 0) RELATIVE mono
sample_monitor PSD_monitor        AT      (0, 0, 0.01)          RELATIVE sample_slit
sample_slit    Slit               AT      (0, 0, 1.0)           RELATIVE beam_dir   
sample         PowderN            AT      (0, 0, 1.1)           RELATIVE beam_dir   
banana         Monitor_nD         AT      (0, 0, 0)             RELATIVE sample     
monitor        PSD_monitor        AT      (0, 0, 0.1)           RELATIVE sample     


In [56]:
mon.set_AT(1.0, RELATIVE="beam_dir")
slit.set_AT(0.01, RELATIVE=mon)

In [57]:
try:
    instrument.check_for_errors()
except Exception as e:
    print(e)

### Running the instrument to check
The sample monitor is now in front of the slit and everything should be working again.

In [60]:
ms_widget.show(instrument)

## Removing a component
Use the *remove_component* to remove a component from the instrument.

In [63]:
instrument.remove_component(slit)
instrument.remove_component("sample_monitor")
ms_widget.show(instrument)

## Final remarks
McStasScript provides a new way to use McStas from python!

A full tutorial exist as Jupyter Notebooks, this presentation was a condensed version of the first part called McStas_basics.
The tutorial covers both how to use McStasScript and the Union components.

Documentaton and tutorials can be found here [https://mads-bertelsen.github.io](https://mads-bertelsen.github.io)

## Exercise

On the e-learning platform go to the jupyter lab and navigate to the McStas Advanved School day 1 directory, here you will find this talk as a jupyter notebook and todays exercises along with solutions. After this talk you are ready for exercise 1!